In [1]:
from datetime import datetime
from sqlalchemy import (MetaData, Table, Column, Integer, Numeric, String,
        DateTime, ForeignKey, Boolean, create_engine, insert)

## Schema

In [6]:
class DataAccessLayer:
        
    def __init__(self,schema=None):        
        self.connection = None
        self.engine = None
        self.conn_string = None
        self.metadata = MetaData()
        
        self.cookies = Table('cookies', self.metadata,
            Column('cookie_id', Integer()),
            Column('cookie_name', String(50), index=True),
            Column('cookie_recipe_url', String(255)),
            Column('cookie_sku', String(55)),
            Column('quantity', Integer()),
            Column('unit_cost', Numeric(12, 2)),
            schema=schema,
        )
    
        self.users = Table('users', self.metadata,
            Column('user_id', Integer()),
            Column('customer_number', Integer(), autoincrement=True),
            Column('username', String(15), nullable=False, unique=True),
            Column('email_address', String(255), nullable=False),
            Column('phone', String(20), nullable=False),
            Column('password', String(25), nullable=False),
            schema=schema,
        )
    
        self.orders = Table('orders', self.metadata,
            Column('order_id', String(20)),
            Column('user_id' , Integer()),
            Column('shipped', Boolean(), default=False),
            schema=schema,                            
        )
    
        self.line_items = Table('line_items', self.metadata,
            Column('line_items_id', Integer()),
            Column('order_id', String(20)),
            Column('cookie_id', Integer()),
            Column('quantity', Integer()),
            Column('extended_cost', Numeric(12, 2)),
            schema=schema,
        )
    
    def db_init(self, conn_string):
        self.engine = create_engine(conn_string or self.conn_string)
        self.metadata.create_all(self.engine)
        self.connection = self.engine.connect()

## Data

In [3]:
data = {

'cookies' : [
    {        
        'cookie_id' : 1,
        'cookie_name': 'dark chocolate chip',
        'cookie_recipe_url': 'http://some.aweso.me/cookie/recipe_dark.html',
        'cookie_sku': 'CC02',
        'quantity': '1',
        'unit_cost' : '0.75'
    },
    {             
        'cookie_id' : 2,
        'cookie_name': 'peanut butter',
        'cookie_recipe_url': 'http://some.aweso.me/cookie/peanut.html',
        'cookie_sku': 'PB01',
        'quantity': '24',
        'unit_cost': '0.25'
    },
    {
        'cookie_id' : 3,
        'cookie_name': 'oatmeal raisin',
        'cookie_recipe_url': 'http://some.okay.me/cookie/raisin.html',
        'cookie_sku': 'EWW01',
        'quantity': '100',
        'unit_cost': '1.00'
    }
] ,   

'users' : [
    {
        'user_id':1,
        'username': 'cookiemon',
        'email_address': "mon@cookie.com",
        'phone': "111-111-1111",
        'password': "password"
    },
    {
        'user_id':2,
        'username': "cakeeater",
        'email_address': "cakeeater@cake.com",
        'phone': "222-222-2222",
        'password': "password"
    },
    {
        'user_id':3,
        'username': "pieguy",
        'email_address': "guy@pie.com",
        'phone': "333-333-3333",
        'password': "password"
    }
],

'orders' : [
    {
         'user_id':1, 
         'order_id':'wlk001'   
    },
    {
        'user_id':2, 
        'order_id':'ol001'
    }
],

'line_items' : [
    {
        'order_id': 'wlk001',
        'cookie_id': 1,
        'quantity': 2,
        'extended_cost': 1.00
    },
    {
        'order_id': 'wlk001',
        'cookie_id': 3,
        'quantity': 12,
        'extended_cost': 3.00
    },
    {
    'order_id': 'ol001',
    'cookie_id': 1,
    'quantity': 24,
    'extended_cost': 12.00
    },
    {
    'order_id': 'ol001',
    'cookie_id': 2,
    'quantity': 6,
    'extended_cost': 6.00
    }
]
 
}

In [4]:
def db_populate(dal,data):
    for table,values in data.items():
        if dal.metadata.tables[table].schema:
            table = dal.metadata.tables[table].schema+'.'+table        
        dal.connection.execute(dal.metadata.tables[table].insert(),values)

## Application

In [5]:
from sqlalchemy.sql import select

def get_orders_by_customer(dal,cust_name, shipped=None, details=False):
    columns = [dal.orders.c.order_id,
               dal.users.c.username,
               dal.users.c.phone]
    joins = dal.users.join(dal.orders,
        onclause=dal.users.c.user_id==dal.orders.c.user_id)
    if details:
        columns.extend([dal.cookies.c.cookie_name,
                        dal.line_items.c.quantity,
                        dal.line_items.c.extended_cost])
        joins = joins.join(dal.line_items,
            onclause=dal.orders.c.order_id==dal.line_items.c.order_id)
        joins = joins.join(dal.cookies,
            onclause=dal.line_items.c.cookie_id==dal.cookies.c.cookie_id)

    cust_orders = select(columns)
    cust_orders = cust_orders.select_from(joins).where(
        dal.users.c.username == cust_name)
    if shipped is not None:
        cust_orders = cust_orders.where(dal.orders.c.shipped == shipped)
    result = dal.connection.execute(cust_orders).fetchall()
    return result

## Unit Tests

In [8]:
import unittest
from decimal import Decimal

class TestApp(unittest.TestCase):
    cookie_orders = [(u'wlk001', u'cookiemon', u'111-111-1111')]
    cookie_details = [
        (u'wlk001', u'cookiemon', u'111-111-1111',
            u'dark chocolate chip', 2, Decimal('1.00')),
        (u'wlk001', u'cookiemon', u'111-111-1111',
            u'oatmeal raisin', 12, Decimal('3.00'))]
    dal = DataAccessLayer()
    
    @classmethod
    def setUpClass(cls):      
        cls.dal.db_init('sqlite:///:memory:')     
        db_populate(cls.dal,data)
       
    @classmethod
    def tearDownClass(cls):
        cls.dal.metadata.drop_all(cls.dal.engine)
        cls.dal.connection.close()
        cls.dal.engine.dispose()

    def test_orders_by_customer_blank(self):
        results = get_orders_by_customer(self.dal,'')
        self.assertEqual(results, [])

    def test_orders_by_customer_blank_shipped(self):
        results = get_orders_by_customer(self.dal,'', True)
        self.assertEqual(results, [])

    def test_orders_by_customer_blank_notshipped(self):
        results = get_orders_by_customer(self.dal,'', False)
        self.assertEqual(results, [])

    def test_orders_by_customer_blank_details(self):
        results = get_orders_by_customer(self.dal,'', details=True)
        self.assertEqual(results, [])

    def test_orders_by_customer_blank_shipped_details(self):
        results = get_orders_by_customer(self.dal,'', True, True)
        self.assertEqual(results, [])

    def test_orders_by_customer_blank_notshipped_details(self):
        results = get_orders_by_customer(self.dal,'', False, True)
        self.assertEqual(results, [])        

    def test_orders_by_customer_bad_cust(self):
        results = get_orders_by_customer(self.dal,'bad name')
        self.assertEqual(results, [])

    def test_orders_by_customer_bad_cust_shipped(self):
        results = get_orders_by_customer(self.dal,'bad name', True)
        self.assertEqual(results, [])

    def test_orders_by_customer_bad_cust_notshipped(self):
        results = get_orders_by_customer(self.dal,'bad name', False)
        self.assertEqual(results, [])

    def test_orders_by_customer_bad_cust_details(self):
        results = get_orders_by_customer(self.dal,'bad name', details=True)
        self.assertEqual(results, [])

    def test_orders_by_customer_bad_cust_shipped_details(self):
        results = get_orders_by_customer(self.dal,'bad name', True, True)
        self.assertEqual(results, [])

    def test_orders_by_customer_bad_cust_notshipped_details(self):
        results = get_orders_by_customer(self.dal,'bad name', False, True)
        self.assertEqual(results, [])

    def test_orders_by_customer(self):
        results = get_orders_by_customer(self.dal,'cookiemon')
        self.assertEqual(results, self.cookie_orders)

    def test_orders_by_customer_shipped_only(self):
        results = get_orders_by_customer(self.dal,'cookiemon', True)
        self.assertEqual(results, [])

    def test_orders_by_customer_unshipped_only(self):
        results = get_orders_by_customer(self.dal,'cookiemon', False)
        self.assertEqual(results, self.cookie_orders)

    def test_orders_by_customer_with_details(self):
        results = get_orders_by_customer(self.dal,'cookiemon', details=True)
        self.assertEqual(results, self.cookie_details)

    def test_orders_by_customer_shipped_only_with_details(self):
        results = get_orders_by_customer(self.dal,'cookiemon', True, True)
        self.assertEqual(results, [])

    def test_orders_by_customer_unshipped_only_details(self):
        results = get_orders_by_customer(self.dal,'cookiemon', False, True)
        self.assertEqual(results, self.cookie_details)

In [9]:
unittest.main(argv=[''], verbosity=2, exit=False);

test_orders_by_customer (__main__.TestApp) ... ok
test_orders_by_customer_bad_cust (__main__.TestApp) ... ok
test_orders_by_customer_bad_cust_details (__main__.TestApp) ... c:\users\soumaya\pycharmprojects\sqlalchemy\venv\lib\site-packages\sqlalchemy\sql\sqltypes.py:665: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  "storage." % (dialect.name, dialect.driver)
ok
test_orders_by_customer_bad_cust_notshipped (__main__.TestApp) ... ok
test_orders_by_customer_bad_cust_notshipped_details (__main__.TestApp) ... ok
test_orders_by_customer_bad_cust_shipped (__main__.TestApp) ... ok
test_orders_by_customer_bad_cust_shipped_details (__main__.TestApp) ... ok
test_orders_by_customer_blank (__main__.TestApp) ... ok
test_orders_by_customer_blank_details (__main__.TestApp) 